In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r
import pandas_datareader as pdr
from dotenv import load_dotenv
import os
load_dotenv()
token = os.getenv("ALPHA")
from time import sleep

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2022-04-28 2022-04-30


In [5]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
        test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                , start=start
                                                , end=end
                                                , retry_count=3, pause=0.1, session=None, adjust_price=True)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = ticker
        panda_reader_data.reset_index(inplace=True)
        market.store("pdr_prices",panda_reader_data)
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
        stuff = r.get(url,params={"apikey":token,"symbol":ticker,"outputsize":"compact"})
        data = stuff.json()
        example = pd.DataFrame.from_records(data["Time Series (Daily)"]).T
        for col in example.columns:
            example.rename(columns={col:col.split(".")[1].strip()},inplace=True)
        example.reset_index(inplace=True)
        example.rename(columns={"index":"date"},inplace=True)
        example["ticker"] = ticker
        market.store("alpha_prices",example)
        sleep(12)
    except Exception as e:
        print(str(e))
market.disconnect()

 17%|█████████████████████████████▏                                                                                                                                         | 88/504 [20:57<1:15:52, 10.94s/it]

Expecting value: line 1 column 1 (char 0)


 29%|████████████████████████████████████████████████▋                                                                                                                     | 148/504 [36:35<1:26:39, 14.60s/it]

market prices documents must be a non-empty list


 30%|█████████████████████████████████████████████████                                                                                                                     | 149/504 [36:37<1:03:07, 10.67s/it]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 503/504 [1:59:33<00:04,  4.36s/it]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 504/504 [1:59:33<00:00, 14.23s/it]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|██████████████████████                                                                                                                                                   | 66/505 [00:04<00:32, 13.57it/s]

'date'


 16%|███████████████████████████▊                                                                                                                                             | 83/505 [00:06<00:24, 16.99it/s]

'date'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:36<00:00, 13.85it/s]

'date'
'date'
'date'
'date'


In [7]:
period = "day"
a = pd.DataFrame(analysis)
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker
318,-0.233245,0.135383,0.135383,0.100827,0.078605,MHK
147,-0.035531,-0.023581,-0.023581,-0.053101,0.058951,DISCA
93,-0.122694,0.022689,0.022689,0.035445,0.028056,CE
233,-0.064265,-0.012855,-0.012855,0.008810,0.018903,HON
470,0.227790,0.026564,0.026564,0.005668,0.017049,VRTX
488,-0.202075,0.096714,0.096714,0.040384,0.008744,WDC
231,-0.015185,-0.064214,-0.064214,-0.010175,0.008263,HOLX
88,-0.272552,-0.205962,-0.205962,0.004659,0.007341,CTLT
23,0.158432,0.054259,0.054259,0.006156,0.006703,MO
331,0.047038,0.081776,0.081776,-0.005157,0.005211,NWL


In [8]:
a = a.merge(sp5[["Symbol","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,GICS Sector
0,-0.163947,-0.007282,-0.007282,0.000000,-0.003821,MMM,Industrials
1,-0.183688,-0.042840,-0.042840,-0.054245,-0.038217,ABT,Health Care
2,0.084626,-0.097123,-0.097123,-0.060269,-0.060329,ABBV,Health Care
3,-0.217614,-0.153733,-0.153733,-0.051499,-0.002228,ABMD,Health Care
4,-0.260498,-0.118248,-0.118248,-0.051954,-0.042952,ACN,Information Technology
...,...,...,...,...,...,...,...
494,-0.163243,-0.125884,-0.125884,-0.023958,-0.003675,WYNN,Consumer Discretionary
495,0.068876,0.007010,0.007010,-0.005970,-0.021635,XEL,Utilities
496,0.020567,0.020567,-0.015404,0.000000,0.000000,XLNX,Information Technology
497,-0.294124,-0.066450,-0.066450,-0.042237,-0.034888,XYL,Industrials


In [9]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Staples,0.026473,0.017577,0.017577,-0.025779,-0.025929
Energy,0.316311,-0.034228,-0.034228,0.020624,-0.022745
Materials,-0.001276,-0.035132,-0.035132,-0.005282,-0.020344
Utilities,0.013232,-0.049970,-0.049970,-0.037266,-0.029579
Consumer Discretionary,-0.184339,-0.050493,-0.050493,-0.040797,-0.028767
Industrials,-0.093000,-0.058694,-0.057481,-0.030187,-0.028503
Real Estate,-0.077779,-0.059090,-0.059090,-0.050071,-0.046541
Health Care,-0.104348,-0.089204,-0.089204,-0.042966,-0.020952
Financials,-0.114026,-0.091058,-0.090123,-0.047854,-0.035010
